In [2]:
import pandas as pd
#from prepare import basic_clean, tokenize, stem, lemmatize, remove_stopwords, prep_article_data, words
from acquire import scrape_github_data

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

# Import Decision Tree and Random Forest ;D
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)

import warnings
warnings.filterwarnings('ignore')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
df = pd.read_json('data.json')

df = words(df)

#df.info()
#df.head()

### Modeling prep

In [4]:
# Make copies of df with prepared columns and target
clean_df = df.copy()[['language', 'clean']]
stem_df = df.copy()[['language', 'stemmed']]
lem_df = df.copy()[['language', 'lemmatized']]


# Get splits for each of the above dfs and isolate target
X_clean = clean_df[['clean']]
y_clean = clean_df.language

X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X_clean, y_clean, test_size=.2, random_state=302)
X_clean_train, X_clean_validate, y_clean_train, y_clean_validate  = train_test_split(X_clean_train, y_clean_train, test_size=.3, random_state=302)

print(X_clean_train.shape, X_clean_validate.shape, X_clean_test.shape)

X_stem = stem_df[['stemmed']]
y_stem = stem_df.language

X_stem_train, X_stem_test, y_stem_train, y_stem_test = train_test_split(X_stem, y_stem, test_size=.2, random_state=302)
X_stem_train, X_stem_validate, y_stem_train, y_stem_validate  = train_test_split(X_stem_train, y_stem_train, test_size=.3, random_state=302)

print(X_stem_train.shape, X_stem_validate.shape, X_stem_test.shape)

X_lem = lem_df[['lemmatized']]
y_lem = lem_df.language

X_lem_train, X_lem_test, y_lem_train, y_lem_test = train_test_split(X_lem, y_lem, test_size=.2, random_state=302)
X_lem_train, X_lem_validate, y_lem_train, y_lem_validate  = train_test_split(X_lem_train, y_lem_train, test_size=.3, random_state=302)

print(X_lem_train.shape, X_lem_validate.shape, X_lem_test.shape)

(280, 1) (120, 1) (100, 1)
(280, 1) (120, 1) (100, 1)
(280, 1) (120, 1) (100, 1)


### Models

In [ ]:
# Make vectorizer objects for bags of words (clean_df)
cv_clean = CountVectorizer()
tfidf_clean = TfidfVectorizer()

#Bags of words
cv_clean_bow = cv_clean.fit_transform(X_clean_train.clean)
tf_clean_bow = tfidf_clean.fit_transform(X_clean_train.clean)

# Make and fit decision tree object for cv_clean_bow
cv_tree1 = DecisionTreeClassifier(max_depth=5)
cv_tree1.fit(cv_clean_bow, y_clean_train)

# #Output tree score
# print(f'CV tree 1 score: {cv_tree1.score(cv_clean_bow, y_clean_train)}')

# #Make and fit decision tree object for tf_clean_bow
# tf_tree1 = DecisionTreeClassifier(max_depth=5)
# tf_tree1.fit(tf_clean_bow, y_clean_train)

#Output tree score
#print(f'TF/IDF tree 1 score: {tf_tree1.score(tf_clean_bow, y_clean_train)}')

In [5]:
# # "Stemmed" models
# cv_stem = CountVectorizer()
# tfidf_stem = TfidfVectorizer()

# # Bags
# cv_stem_bow = cv_stem.fit_transform(X_stem_train.stemmed)
# tf_stem_bow = tfidf_stem.fit_transform(X_stem_train.stemmed)

# # Make and fit decision tree object for cv_stem_bow
# cv_tree2 = DecisionTreeClassifier(max_depth=5)
# cv_tree2.fit(cv_stem_bow, y_stem_train)

# #Output tree score
# print(f'CV tree 2 score: {cv_tree2.score(cv_stem_bow, y_stem_train)}')

# #Make and fit decision tree object for tf_stem_bow
# tf_tree2 = DecisionTreeClassifier(max_depth=5)
# tf_tree2.fit(tf_stem_bow, y_stem_train)

# #Output tree score
# print(f'TF IDF tree 2 score: {tf_tree2.score(tf_stem_bow, y_stem_train)}')

CV tree 2 score: 0.8571428571428571
TF IDF tree 2 score: 0.8928571428571429


In [6]:
# # "Lemmatized" models
# cv_lem = CountVectorizer()
# tfidf_lem = TfidfVectorizer()

# cv_lem_bow = cv_lem.fit_transform(X_lem_train.lemmatized)
# tf_lem_bow = tfidf_lem.fit_transform(X_lem_train.lemmatized)

# # Make and fit decision tree object for cv_lem_bow
# cv_tree3 = DecisionTreeClassifier(max_depth=5)
# cv_tree3.fit(cv_lem_bow, y_lem_train)

# #Make and fit decision tree object for tf_lem_bow
# tf_tree3 = DecisionTreeClassifier(max_depth=5)
# tf_tree3.fit(tf_lem_bow, y_lem_train)

# Output tree scores
print(f'CV tree 3 score: {cv_tree3.score(cv_lem_bow, y_lem_train)}') 
print(f'TFIDF tree 3 score: {tf_tree3.score(tf_lem_bow, y_lem_train)}')

CV tree 3 score: 0.8571428571428571
TFIDF tree 3 score: 0.9285714285714286


In [6]:
def classi_bow(X_lem_train,y_lem_train,X_stem_train,y_stem_train,X_clean_train,y_clean_train):

    # Make vectorizer objects for bags of words (clean_df)
    cv_clean = CountVectorizer()
    tfidf_clean = TfidfVectorizer()

    #Bags of words
    cv_clean_bow = cv_clean.fit_transform(X_clean_train[['clean']].clean)
    tf_clean_bow = tfidf_clean.fit_transform(X_clean_train[['clean']].clean)
    
    # Make and fit decision tree object for cv_clean_bow
    cv_tree1 = DecisionTreeClassifier(max_depth=5)
    cv_tree1.fit(cv_clean_bow, y_clean_train)

    #Make and fit decision tree object for tf_clean_bow
    tf_tree1 = DecisionTreeClassifier(max_depth=5)
    tf_tree1.fit(tf_clean_bow, y_clean_train)


    # "Stemmed" models
    cv_stem = CountVectorizer()
    tfidf_stem = TfidfVectorizer()

    # Bags
    cv_stem_bow = cv_stem.fit_transform(X_stem_train[['stemmed']].stemmed)
    tf_stem_bow = tfidf_stem.fit_transform(X_stem_train[['stemmed']].stemmed)

    # Make and fit decision tree object for cv_stem_bow
    cv_tree2 = DecisionTreeClassifier(max_depth=5)
    cv_tree2.fit(cv_stem_bow, y_stem_train)

    # Make and fit decision tree object for tf_stem_bow
    tf_tree2 = DecisionTreeClassifier(max_depth=5)
    tf_tree2.fit(tf_stem_bow, y_stem_train)


    # "Lemmatized" models
    cv_lem = CountVectorizer()
    tfidf_lem = TfidfVectorizer()

    # Bags
    cv_lem_bow = cv_lem.fit_transform(X_lem_train[['lemmatized']].lemmatized)
    tf_lem_bow = tfidf_lem.fit_transform(X_lem_train[['lemmatized']].lemmatized)

    # Make and fit decision tree object for cv_lem_bow
    cv_tree3 = DecisionTreeClassifier(max_depth=5)
    cv_tree3.fit(cv_lem_bow, y_lem_train)

    #Make and fit decision tree object for tf_lem_bow
    tf_tree3 = DecisionTreeClassifier(max_depth=5)
    tf_tree3.fit(tf_lem_bow, y_lem_train)


    dec_tree_training_scores=pd.Series({
        'CV_clean': cv_tree1.score(cv_clean_bow, y_clean_train),
        'CV_stem': cv_tree2.score(cv_stem_bow, y_stem_train),
        'CV_lem': cv_tree3.score(cv_lem_bow, y_lem_train),
        'TFIDF_clean': tf_tree1.score(tf_clean_bow, y_clean_train),
        'TFIDF_stem': tf_tree2.score(tf_stem_bow, y_stem_train),
        'TFIDF_lem': tf_tree3.score(tf_lem_bow, y_lem_train)
    })

    return dec_tree_training_scores

In [7]:
dec_tree_training_scores=classi_bow(X_lem_train,y_lem_train,X_stem_train,y_stem_train,X_clean_train,y_clean_train)

In [8]:
dec_tree_training_scores

CV_clean       0.825000
CV_stem        0.817857
CV_lem         0.821429
TFIDF_clean    0.842857
TFIDF_stem     0.850000
TFIDF_lem      0.839286
dtype: float64

### Training split model evaluation: TF IDF vectorizers outscore count vectorizers. Lemmatized text preparation yields highest score on trees using TF IDF vectorizer.

### Validate

In [ ]:
#Transform validate split with vectorizer
tf_clean_bow_val = tfidf_clean.transform(X_clean_validate.clean)

#Get tf_tree1 score on validate
#tf_tree1.score(tf_clean_bow_val, y_clean_validate)

### Considerable drop off. Probably overfit.

In [10]:
#Transform validate split with vectorizer
tf_stem_bow_val = tfidf_stem.transform(X_stem_validate.stemmed)

#Get tf_tree1 score on validate
#tf_tree2.score(tf_stem_bow_val, y_stem_validate)

0.375

### Even worse dropoff...

In [8]:
#Transform validate split with vectorizer
tf_lem_bow_val = tfidf_lem.transform(X_lem_validate.lemmatized)

#Get tf_tree1 score on validate
#tf_tree3.score(tf_lem_bow_val, y_lem_validate)

0.5416666666666666

### Significant dropoff for each text preparation. Cleaned preparation scored 62.5% and will be tested.

### Random Forest models, 100 estimators, max depth 15

In [10]:
# Make vectorizer objects
cv_clean2 = CountVectorizer()
tfidf_clean2 = TfidfVectorizer()

#Fit vectorizer objects
cv_clean2_bow = cv_clean2.fit_transform(X_clean_train.clean)
tf_clean2_bow = tfidf_clean2.fit_transform(X_clean_train.clean)

# Make and fit decision tree object for cv_clean2_bow
cv_rf1 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf1.fit(cv_clean2_bow, y_clean_train)

#Make and fit decision rf object for tf_clean2_bow
tf_rf1 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf1.fit(tf_clean2_bow, y_clean_train)

#Output rf scores
print(f'CV RF 1 score: {cv_rf1.score(cv_clean2_bow, y_clean_train)}')
print(f'TF/IDF RF 1 score: {tf_rf1.score(tf_clean2_bow, y_clean_train)}')

# Make vectorizer objects
cv_stem2 = CountVectorizer()
tfidf_stem2 = TfidfVectorizer()

#Fit vectorizer objects
cv_stem2_bow = cv_stem2.fit_transform(X_stem_train.stemmed)
tf_stem2_bow = tfidf_stem2.fit_transform(X_stem_train.stemmed)

# Make and fit decision tree object for cv_stem2_bow
cv_rf2 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf2.fit(cv_stem2_bow, y_stem_train)

#Make and fit decision rf object for tf_stem2_bow
tf_rf2 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf2.fit(tf_stem2_bow, y_stem_train)

#Output rf scores
print(f'CV RF 2 score: {cv_rf2.score(cv_stem2_bow, y_stem_train)}')
print(f'TF/IDF RF 2 score: {tf_rf2.score(tf_stem2_bow, y_stem_train)}')
# Make vectorizer objects
cv_lem2 = CountVectorizer()
tfidf_lem2 = TfidfVectorizer()

#Fit vectorizer objects
cv_lem2_bow = cv_lem2.fit_transform(X_lem_train.lemmatized)
tf_lem2_bow = tfidf_lem2.fit_transform(X_lem_train.lemmatized)

# Make and fit decision tree object for cv_lem2_bow
cv_rf3 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf3.fit(cv_lem2_bow, y_lem_train)

#Make and fit decision rf object for tf_lem2_bow
tf_rf3 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf3.fit(tf_lem2_bow, y_lem_train)

#Output rf scores
print(f'CV RF 3 score: {cv_rf3.score(cv_lem2_bow, y_lem_train)}')
print(f'TF/IDF RF 3 score: {tf_rf3.score(tf_lem2_bow, y_lem_train)}')

CV RF 1 score: 0.8071428571428572
TF/IDF RF 1 score: 0.85
CV RF 2 score: 0.8285714285714286
TF/IDF RF 2 score: 0.8785714285714286
CV RF 3 score: 0.8035714285714286
TF/IDF RF 3 score: 0.8535714285714285


### Both vectorizers scoring the same on training split for all text preparations. All move on to validate.

### Update (16-May-2022): TF/IDF vectorizers scoring higher on increased dataset.

In [11]:
# Transform count vectorizers on validate
#cv_clean2_bow_val = cv_clean2.transform(X_clean_validate.clean)
#cv_stem2_bow_val = cv_stem2.transform(X_stem_validate.stemmed)
#cv_lem2_bow_val = cv_lem2.transform(X_lem_validate.lemmatized)

# Output CV RF scores
#print(f'CV RF 1 score on validate: {cv_rf1.score(cv_clean2_bow_val, y_clean_validate)}')
#print(f'CV RF 2 score on validate: {cv_rf2.score(cv_stem2_bow_val, y_stem_validate)}')
#print(f'CV RF 3 score on validate: {cv_rf3.score(cv_lem2_bow_val, y_lem_validate)}')

# Transform TF IDF vectorizers on validate
tf_clean2_bow_val = tfidf_clean2.transform(X_clean_validate.clean)
tf_stem2_bow_val = tfidf_stem2.transform(X_stem_validate.stemmed)
tf_lem2_bow_val = tfidf_lem2.transform(X_lem_validate.lemmatized)

# Output TF IDF RF scores
print(f'TF/IDF RF 3 score on validate: {tf_rf1.score(tf_clean2_bow_val, y_clean_validate)}')
print(f'TF/IDF RF 3 score on validate: {tf_rf2.score(tf_stem2_bow_val, y_stem_validate)}')
print(f'TF/IDF RF 3 score on validate: {tf_rf3.score(tf_lem2_bow_val, y_lem_validate)}')

TF/IDF RF 3 score on validate: 0.675
TF/IDF RF 3 score on validate: 0.6916666666666667
TF/IDF RF 3 score on validate: 0.675


### Lemmatized text prep for both vectorizers score highest, but TF/IDF outscores CV. Significant drop off for both vectorizers on all text preparations, though. 

### Update (16-May-2022): TF/IDF vectorizer on stemmed text scores highest at 69.2%

### Naive Bayes modeling, alpha 0.5

### Hold off on running the below cell on increased dataset for now

In [16]:
from sklearn.naive_bayes import CategoricalNB

# Make vectorizer objects
cv_clean3 = CountVectorizer()
tfidf_clean3 = TfidfVectorizer()

#Fit vectorizer objects
cv_clean3_bow = cv_clean3.fit_transform(X_clean_train.clean)
tf_clean3_bow = tfidf_clean3.fit_transform(X_clean_train.clean)

# Make and fit Naive Bayes object for cv_clean3_bow
cv_nb1 = CategoricalNB(alpha=0.5, min_categories=cv_clean3_bow.toarray().shape[1]) # min_categories kwarg to ensure model is fed a consistent number of categories
cv_nb1.fit(cv_clean3_bow.toarray(), y_clean_train) # Naive Bayes requires dense data

#Make and fit Naive Bayes object for tf_clean3_bow
tf_nb1 = CategoricalNB(alpha=0.5, min_categories=cv_clean3_bow.toarray().shape[1])
tf_nb1.fit(tf_clean3_bow.toarray(), y_clean_train) 

#Output NB scores
print(f'CV NB 1 score: {cv_nb1.score(cv_clean3_bow.toarray(), y_clean_train)}')
print(f'TF IDF NB 1 score: {tf_nb1.score(tf_clean3_bow.toarray(), y_clean_train)}')

# Make vectorizer objects
cv_stem3 = CountVectorizer()
tfidf_stem3 = TfidfVectorizer()

#Fit vectorizer objects
cv_stem3_bow = cv_stem3.fit_transform(X_stem_train.stemmed)
tf_stem3_bow = tfidf_stem3.fit_transform(X_stem_train.stemmed)

# Make and fit Naive Bayes object for cv_stem3_bow
cv_nb2 = CategoricalNB(alpha=0.5, min_categories=cv_stem3_bow.toarray().shape[1])
cv_nb2.fit(cv_stem3_bow.toarray(), y_stem_train)

#Make and fit Naive Bayes object for tf_stem3_bow
tf_nb2 = CategoricalNB(alpha=0.5, min_categories=cv_stem3_bow.toarray().shape[1])
tf_nb2.fit(tf_stem3_bow.toarray(), y_stem_train)

#Output NB scores
print(f'CV NB 2 score: {cv_nb2.score(cv_stem3_bow.toarray(), y_stem_train)}')
print(f'TF IDF NB 2 score: {tf_nb2.score(tf_stem3_bow.toarray(), y_stem_train)}')
# Make vectorizer objects
cv_lem3 = CountVectorizer()
tfidf_lem3 = TfidfVectorizer()

#Fit vectorizer objects
cv_lem3_bow = cv_lem3.fit_transform(X_lem_train.lemmatized)
tf_lem3_bow = tfidf_lem3.fit_transform(X_lem_train.lemmatized)

# Make and fit Naive Bayes object for cv_lem3_bow
cv_nb3 = CategoricalNB(alpha=0.5, min_categories=cv_lem3_bow.toarray().shape[1])
cv_nb3.fit(cv_lem3_bow.toarray(), y_lem_train)

#Make and fit Naive Bayes object for tf_lem3_bow
tf_nb3 = CategoricalNB(alpha=0.5, min_categories=cv_lem3_bow.toarray().shape[1])
tf_nb3.fit(tf_lem3_bow.toarray(), y_lem_train)

#Output NB scores
print(f'CV NB 3 score: {cv_nb3.score(cv_lem3_bow.toarray(), y_lem_train)}')
print(f'TF IDF NB 3 score: {tf_nb3.score(tf_lem3_bow.toarray(), y_lem_train)}')


CV NB 1 score: 0.5357142857142857
TF IDF NB 1 score: 0.5357142857142857
CV NB 2 score: 0.5357142857142857
TF IDF NB 2 score: 0.5357142857142857
CV NB 3 score: 0.5357142857142857
TF IDF NB 3 score: 0.5357142857142857


### Naive Bayes models score significantly lower than DT and RF on training data. Equal scores across text preparations and vectorizers.

### Hold off on running the below cell on increased dataset for now

In [17]:
# Transform count vectorizers on validate
cv_clean3_bow_val = cv_clean3.transform(X_clean_validate.clean)
cv_stem3_bow_val = cv_stem3.transform(X_stem_validate.stemmed)
cv_lem3_bow_val = cv_lem3.transform(X_lem_validate.lemmatized)

tf_clean3_bow_val = tfidf_clean3.transform(X_clean_validate.clean)
tf_stem3_bow_val = tfidf_stem3.transform(X_stem_validate.stemmed)
tf_lem3_bow_val = tfidf_lem3.transform(X_lem_validate.lemmatized)

# Output CV NB scores
print(f'CV NB 1 score on validate: {cv_nb1.score(cv_clean3_bow_val.toarray(), y_clean_validate)}')
print(f'CV NB 2 score on validate: {cv_nb2.score(cv_stem3_bow_val.toarray(), y_stem_validate)}')
print(f'CV NB 3 score on validate: {cv_nb3.score(cv_lem3_bow_val.toarray(), y_lem_validate)}')

# Output TF/IDF NB scores
print(f'TF/IDF NB 1 score on validate: {tf_nb1.score(tf_clean3_bow_val.toarray(), y_clean_validate)}')
print(f'TF/IDF NB 2 score on validate: {tf_nb2.score(tf_stem3_bow_val.toarray(), y_stem_validate)}')
print(f'TF/IDF NB 3 score on validate: {tf_nb3.score(tf_lem3_bow_val.toarray(), y_lem_validate)}')

CV NB 1 score on validate: 0.5833333333333334
CV NB 2 score on validate: 0.5833333333333334
CV NB 3 score on validate: 0.5833333333333334
TF/IDF NB 1 score on validate: 0.5833333333333334
TF/IDF NB 2 score on validate: 0.5833333333333334
TF/IDF NB 3 score on validate: 0.5833333333333334


### Incremental improvement. Scores still even across all text preparations. 

### Bags of bigrams -- Decision Trees

In [12]:

# Make bags for bigrams
cv_clean4 = CountVectorizer(ngram_range=(2,2))
tfidf_clean4 = TfidfVectorizer(ngram_range=(2,2))

# Fill bags with words
cv_clean4_bow = cv_clean4.fit_transform(X_clean_train.clean)
tf_clean4_bow = tfidf_clean4.fit_transform(X_clean_train.clean)


# Make and fit decision tree object for cv_clean4_bow
cv_tree4 = DecisionTreeClassifier(max_depth=5)
cv_tree4.fit(cv_clean4_bow, y_clean_train)

#Make and fit decision tree object for tf_clean_bow
tf_tree4 = DecisionTreeClassifier(max_depth=5)
tf_tree4.fit(tf_clean4_bow, y_clean_train)

#Output tree scores
print(f'CV tree 4 score: {cv_tree4.score(cv_clean4_bow, y_clean_train)}')
print(f'TF/IDF tree 4 score: {tf_tree4.score(tf_clean4_bow, y_clean_train)}')

CV tree 4 score: 0.7214285714285714
TF/IDF tree 4 score: 0.7214285714285714


In [13]:
#Make bags
cv_stem4 = CountVectorizer(ngram_range=(2,2))
tfidf_stem4 = TfidfVectorizer(ngram_range=(2,2))

# Fill bags with words
cv_stem4_bow = cv_stem4.fit_transform(X_stem_train.stemmed)
tf_stem4_bow = tfidf_stem4.fit_transform(X_stem_train.stemmed)


# Make and fit decision tree object for cv_stem4_bow
cv_tree5 = DecisionTreeClassifier(max_depth=5)
cv_tree5.fit(cv_stem4_bow, y_stem_train)

#Make and fit decision tree object for tf_stem_bow
tf_tree5 = DecisionTreeClassifier(max_depth=5)
tf_tree5.fit(tf_stem4_bow, y_stem_train)

#Output tree scores
print(f'CV tree 5 score: {cv_tree5.score(cv_stem4_bow, y_stem_train)}')
print(f'TF/IDF tree 5 score: {tf_tree5.score(tf_stem4_bow, y_stem_train)}')

CV tree 5 score: 0.725
TF/IDF tree 5 score: 0.7285714285714285


In [14]:
# Make bags
cv_lem4 = CountVectorizer(ngram_range=(2,2))
tfidf_lem4 = TfidfVectorizer(ngram_range=(2,2))

# Fill bags with words
cv_lem4_bow = cv_lem4.fit_transform(X_lem_train.lemmatized)
tf_lem4_bow = tfidf_lem4.fit_transform(X_lem_train.lemmatized)


# Make and fit decision tree object for cv_lem4_bow
cv_tree6 = DecisionTreeClassifier(max_depth=5)
cv_tree6.fit(cv_lem4_bow, y_lem_train)

#Make and fit decision tree object for tf_lem_bow
tf_tree6 = DecisionTreeClassifier(max_depth=5)
tf_tree6.fit(tf_lem4_bow, y_lem_train)

#Output tree scores
print(f'CV tree 6 score: {cv_tree6.score(cv_lem4_bow, y_lem_train)}')
print(f'TF/IDF tree 6 score: {tf_tree6.score(tf_lem4_bow, y_lem_train)}')

CV tree 6 score: 0.7214285714285714
TF/IDF tree 6 score: 0.725


### TF/IDF vectorizers on stemmed and lemmatized text outperform count vectorizers by a thin margin. Both vectorizer scored the same on clean text.

In [16]:
print('TF/IDF [Bigram bags] Decision Tree scores on validate split:')
print('------------------------------------------------------------')
#Transform validate split with vectorizer
tf_clean4_bow_val = tfidf_clean4.transform(X_clean_validate.clean)

#Output tf_tree4 score on validate
print(f'Tree 4 [clean] score: {tf_tree4.score(tf_clean4_bow_val, y_clean_validate)}')

#Transform validate split with vectorizer
tf_stem4_bow_val = tfidf_stem4.transform(X_stem_validate.stemmed)

#Output tf_tree5 score on validate
print(f'Tree 5 [stemmed] score: {tf_tree5.score(tf_stem4_bow_val, y_stem_validate)}')

#Transform validate split with vectorizer
tf_lem4_bow_val = tfidf_lem4.transform(X_lem_validate.lemmatized)

#Output tf_tree6 score on validate
print(f'Tree 6 [lemmatized] score: {tf_tree6.score(tf_lem4_bow_val, y_lem_validate)}')

TF/IDF [Bigram bags] Decision Tree scores on validate split:
------------------------------------------------------------
Tree 4 [clean] score: 0.7666666666666667
Tree 5 [stemmed] score: 0.7583333333333333
Tree 6 [lemmatized] score: 0.7416666666666667


### Bigram Random Forests

In [17]:
# Make vectorizer objects
cv_clean5 = CountVectorizer(ngram_range=(2,2))
tfidf_clean5 = TfidfVectorizer(ngram_range=(2,2))

#Fit vectorizer objects
cv_clean5_bow = cv_clean5.fit_transform(X_clean_train.clean)
tf_clean5_bow = tfidf_clean5.fit_transform(X_clean_train.clean)

# Make and fit random forest object for cv_clean5_bow
cv_rf4 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf4.fit(cv_clean5_bow, y_clean_train)

#Make and fit random forest object for tf_clean5_bow
tf_rf4 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf4.fit(tf_clean5_bow, y_clean_train)

#Output rf scores
print(f'CV RF 4 score: {cv_rf4.score(cv_clean5_bow, y_clean_train)}')
print(f'TF IDF RF 4 score: {tf_rf4.score(tf_clean5_bow, y_clean_train)}')

# Make vectorizer objects
cv_stem5 = CountVectorizer()
tfidf_stem5 = TfidfVectorizer()

#Fit vectorizer objects
cv_stem5_bow = cv_stem5.fit_transform(X_stem_train.stemmed)
tf_stem5_bow = tfidf_stem5.fit_transform(X_stem_train.stemmed)

# Make and fit random object for cv_stem5_bow
cv_rf5 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf5.fit(cv_stem5_bow, y_stem_train)

#Make and fit decision rf object for tf_stem5_bow
tf_rf5 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf5.fit(tf_stem5_bow, y_stem_train)

#Output rf scores
print(f'CV RF 5 score: {cv_rf5.score(cv_stem5_bow, y_stem_train)}')
print(f'TF IDF RF 5 score: {tf_rf5.score(tf_stem5_bow, y_stem_train)}')
# Make vectorizer objects
cv_lem5 = CountVectorizer()
tfidf_lem5 = TfidfVectorizer()

#Fit vectorizer objects
cv_lem5_bow = cv_lem5.fit_transform(X_lem_train.lemmatized)
tf_lem5_bow = tfidf_lem5.fit_transform(X_lem_train.lemmatized)

# Make and fit decision tree object for cv_lem5_bow
cv_rf6 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf6.fit(cv_lem5_bow, y_lem_train)

#Make and fit decision rf object for tf_lem5_bow
tf_rf6 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf6.fit(tf_lem5_bow, y_lem_train)

#Output rf scores
print(f'CV RF 6 score: {cv_rf6.score(cv_lem5_bow, y_lem_train)}')
print(f'TF IDF RF 6 score: {tf_rf6.score(tf_lem5_bow, y_lem_train)}')

CV RF 4 score: 0.5892857142857143
TF IDF RF 4 score: 0.5714285714285714
CV RF 5 score: 0.8392857142857143
TF IDF RF 5 score: 0.8714285714285714
CV RF 6 score: 0.7928571428571428
TF IDF RF 6 score: 0.8642857142857143


### TF/IDF vectorizers outperform count vectorizers overall. RFs on stemmed bigrams scoring highest.

In [18]:
# Transform count vectorizers on validate
cv_stem5_bow_val = cv_stem5.transform(X_stem_validate.stemmed)
cv_lem5_bow_val = cv_lem5.transform(X_lem_validate.lemmatized)

# Transform TF/IDF vectorizers on validate
tf_stem5_bow_val = tfidf_stem5.transform(X_stem_validate.stemmed)
tf_lem5_bow_val = tfidf_lem5.transform(X_lem_validate.lemmatized)

#Output CV RF 5 & 6 scores on validate
print(f'CV RF 5 score on validate: {cv_rf5.score(cv_stem5_bow_val, y_stem_validate)}')
print(f'CV RF 6 score on validate: {cv_rf6.score(cv_lem5_bow_val, y_lem_validate)}')

# Output TF/IDF RF 5 & 6 scores on validate
print(f'TF/IDF RF 5 score on validate: {tf_rf5.score(tf_stem5_bow_val, y_stem_validate)}')
print(f'TF/IDF RF 6 score on validate: {tf_rf6.score(tf_lem5_bow_val, y_lem_validate)}')

CV RF 5 score on validate: 0.6833333333333333
CV RF 6 score on validate: 0.6833333333333333
TF/IDF RF 5 score on validate: 0.6666666666666666
TF/IDF RF 6 score on validate: 0.6666666666666666


### About 20% dropoff in accuracy on validate. Count Vectorizers performing slightly better than TF/IDF.